### Retrieval-based Chatbot building with help of Natural Language Processing (NLP) using NLTK and Deep Learning.



In [1]:
# Libraries needed for NLP
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import json

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [2]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore = ['?',"'s",'!','.']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Perform lemmation and lower each word as well as remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

43 documents
9 classes ['developers', 'doctorstype', 'fun', 'goodbye', 'greeting', 'hours', 'noanswer', 'options', 'thanks']
66 unique stemmed words ['123', '555', 'again', 'anyone', 'are', 'be', 'bye', 'byeee', 'can', 'close', 'could', 'day', 'developer', 'do', 'doctor', 'ffff', 'go', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'hour', 'how', 'i', 'is', 'it', 'kind', 'later', 'listed', 'made', 'me', 'meet', 'new', 'nice', 'nnnn', 'of', 'offered', 'open', 'origin', 'provide', 'real', 'really', 's', 'see', 'speciality', 'sup', 'support', 'thank', 'thanks', 'that', 'there', 'to', 'u', 'up', 'what', 'when', 'which', 'who', 'will', 'yo', 'you', 'your']


In [4]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [lemmatizer.lemmatize(w.lower()) for w in pattern_words]
    # create bag of words array

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\HARTE\AppData\Local\Temp/ipykernel_27416/311820709.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

c:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
9/9 [==============================] - 2s 3ms/step - loss: 2.2454 - accuracy: 0.0698
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 2.1562 - accuracy: 0.1860
Epoch 3/100
9/9 [==============================] - 0s 4ms/step - loss: 2.2185 - accuracy: 0.0930
Epoch 4/100
9/9 [==============================] - 0s 3ms/step - loss: 2.1892 - accuracy: 0.1163
Epoch 5/100
9/9 [==============================] - 0s 3ms/step - loss: 2.1937 - accuracy: 0.1395
Epoch 6/100
9/9 [==============================] - 0s 3ms/step - loss: 2.2046 - accuracy: 0.1628
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 2.1506 - accuracy: 0.1860
Epoch 8/100
9/9 [==============================] - 0s 4ms/step - loss: 2.1509 - accuracy: 0.1860
Epoch 9/100
9/9 [==============================] - 0s 3ms/step - loss: 2.1628 - accuracy: 0.2093
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 2.1613 - accuracy: 0.1860
Epoch 11/100
9/9 [===========

In [6]:
print(hist.history['loss'][-1],hist.history['accuracy'][-1])


1.5046833753585815 0.5116279125213623


In [7]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )
# restoring all the data structures
data = pickle.load( open("training_data", "rb"))
words = data['words']
classes = data['classes']

with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  

    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [9]:
ERROR_THRESHOLD = 0.30
from keras.models import load_model
model = load_model('chatbot_model.h5')

def classify(sentence):
    # generate probabilities from the model
    p = bow(sentence, words)
    results = model.predict(np.array([p]))[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return random.choice(i['responses'])

            results.pop(0)
    else:
       return "Sorry, can't understand you"

In [10]:
classify('What are you hours of operation?')

[]

In [11]:
classify(' ')

[]

In [12]:
response('What are you hours of operation?')

"Sorry, can't understand you"

In [13]:
response(' ')

"Sorry, can't understand you"

In [14]:
flag=True
print("I am the Doctor Recommender.\nIf you want to exit, type Bye!")

while(flag==True):
    user_response = input("You- ")
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Chatbot : You are welcome...")
        else:
            answer = response(user_response)
            print(answer)
    else:
        flag=False
        print("Chatbot : Bye!!! ")

I am the Doctor Recommender.
If you want to exit, type Bye!
Hi there, how can I help?
Chatbot : Bye!!! 
